In [ ]:
import numpy as np

ops = {'nclust': 30, # number of clusters
       'iPC': np.arange(0,200).astype(np.int32), # number of PCs to use
       'upsamp': 100, # upsampling factor for embedding position
       'sigUp': 1 # standard deviation for upsampling
      }

S = np.load('C:/Users/carse/github/bigdat/spks.npy')
iscell = np.load('C:/Users/carse/github/bigdat/iscell.npy')
S = S[iscell[:,0].astype(np.bool),:]
print(S.shape)

S = S - S.mean(axis=1)[:,np.newaxis]

# compute svd and keep iPC's of data
u,sv,v = np.linalg.svd(S, full_matrices=0)
isort = np.argsort(u[:,0]).astype(np.int32)


iPC = ops['iPC']
S = u[:,iPC] @ np.diag(sv[iPC])
NN,nPC = S.shape
nclust = ops['nclust']
nn = np.floor(NN/nclust) # number of neurons per cluster
iclust = np.zeros((NN,),np.int32)
# initial cluster assignments based on 1st PC weights
iclust[isort] = np.floor(np.arange(0,NN)/nn).astype(np.int32)
iclust[iclust>nclust] = nclust
# annealing schedule for embedding
sig_anneal = np.concatenate((np.linspace(nclust/10,1,50),np.ones((50,),np.float32)), axis=0)

from scipy.ndimage import gaussian_filter1d
from matplotlib import pyplot as plt
% matplotlib inline

t=0
for sig in sig_anneal:
    V = np.zeros((nPC,nclust), np.float32)
    # compute average activity of each cluster
    for j in range(0,nclust):
        iin = iclust==j
        V[:,j] = S[iin,:].sum(axis=0)
    V = gaussian_filter1d(V,sig,axis=1,mode='reflect') # smooth activity across clusters
    V /= ((V**2).sum(axis=0)[np.newaxis,:])**0.5 # normalize columns to unit norm
    cv = S @ V # reproject onto activity across neurons
    # recompute best clusters
    iclust = np.argmax(cv, axis=1)
    cmax = np.amax(cv, axis=1)
    if t>0:
        score = ((cmax - cmax0)**2).sum()
        print(score)
    cmax0 = cmax
    t+=1


def upsampled_kernel(nclust, sig, upsamp):
    xs = np.arange(0,nclust)
    xn = np.linspace(0, nclust-1, nclust * upsamp)
    d0 = (xs[:,np.newaxis] - xs[np.newaxis,:])**2;
    d1 = (xn[:,np.newaxis] - xs[np.newaxis,:])**2;
    K0 = np.exp(-1*d0/sig)
    K1 = np.exp(-1*d1/sig)
    Km = K1 @ np.linalg.inv(K0 + 0.001 * np.eye(nclust));
    return Km

Km = upsampled_kernel(nclust,ops['sigUp'],ops['upsamp'])
iclustup = np.argmax(cv @ Km.T, axis=1)

In [ ]:
plt.figure(figsize=(16,8))
plt.plot(iclustup)
isort = np.argsort(iclustup)
print(iclustup.max())
ssmooth = scipy.stats.zscore(gaussian_filter1d(S[isort,:],10,axis=0),axis=1)
#plt.imshow(Km[:300,:])

In [ ]:
import scipy.stats
plt.figure(figsize=(16,16))
plt.imshow(ssmooth,vmin=0,vmax=3)
plt.show()